# I. $LU$ factorization of a square matrix
When we premultply $A$ by lower triangular elemetary matrices $\Lambda_j$ it is trasfrommed in an  upper triangular matrix $U$

$$  \Lambda_{n-1}  \ldots\Lambda_2  \Lambda_1 A  = U$$

$$  A  = (\Lambda_{n-1}  \ldots\Lambda_2  \Lambda_2 )^{-1} U$$

The inverse of a product of matrices is the revesre product of inverses
$$  A  = (\Lambda_1^{-1}  \Lambda_2^{-1} \ldots  \Lambda_{n-1}^{-1}) U$$

and inverse of a Lower Triangular Elemetary Matrix is minus the matrix $\Lambda_j^{-1} = - \Lambda_j$, so 

$$ A  =  (-\Lambda_1) (-\Lambda_2) \ldots   (-\Lambda_{n-1})  U$$

So the $LU$ column pivot factorization is 
$$  A  = L U$$
with
$$ U = \Lambda_{n-1}  \ldots\Lambda_2  \Lambda_1 A  $$
an upper triangular matrix
$$ L  =  \Lambda_1^{-1}  \Lambda_2^{-1} \ldots  \Lambda_{n-1}^{-1} $$
an lower triangular matrix.

Consider a simple naive implementation of the LU decomposition. 

Note that we're using the `numpy` arrays to represent matrices [do **not** use `np.matrix`].

In [ ]:
import numpy as np

def diy_lu(a):
    """Construct the LU decomposition of the input matrix.
    
    Naive LU decomposition: work column by column, accumulate elementary triangular matrices.
    No pivoting.
    """
    N = a.shape[0]
    
    u = a.copy()
    L = np.eye(N)
    for j in range(N-1):
        lam = np.eye(N)
        gamma = u[j+1:, j] / u[j, j]
        lam[j+1:, j] = -gamma
        u = lam @ u

        lam[j+1:, j] = gamma
        L = L @ lam
    return L, u

In [3]:
# Now, generate a full rank matrix and test the naive implementation

import numpy as np

N = 6
a = np.zeros((N, N), dtype=float)
for i in range(N):
    for j in range(N):
        a[i, j] = 3. / (0.6*i*j + 1)

np.linalg.matrix_rank(a)

6

In [ ]:
a

array([[3.        , 3.        , 3.        , 3.        , 3.        ,
        3.        ],
       [3.        , 1.875     , 1.36363636, 1.07142857, 0.88235294,
        0.75      ],
       [3.        , 1.36363636, 0.88235294, 0.65217391, 0.51724138,
        0.42857143],
       [3.        , 1.07142857, 0.65217391, 0.46875   , 0.36585366,
        0.3       ],
       [3.        , 0.88235294, 0.51724138, 0.36585366, 0.28301887,
        0.23076923],
       [3.        , 0.75      , 0.42857143, 0.3       , 0.23076923,
        0.1875    ]])

In [ ]:
# Tweak the printing of floating-point numbers, for clarity
np.set_printoptions(precision=3)

In [ ]:
L, u = diy_lu(a)

print(L, "\n")
print(u, "\n")

# Quick sanity check: L times U must equal the original matrix, up to floating-point errors.
print(L@u - a)

[[1.    0.    0.    0.    0.    0.   ]
 [1.    1.    0.    0.    0.    0.   ]
 [1.    1.455 1.    0.    0.    0.   ]
 [1.    1.714 1.742 1.    0.    0.   ]
 [1.    1.882 2.276 2.039 1.    0.   ]
 [1.    2.    2.671 2.944 2.354 1.   ]] 

[[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -1.125e+00 -1.636e+00 -1.929e+00 -2.118e+00 -2.250e+00]
 [ 0.000e+00  0.000e+00  2.625e-01  4.574e-01  5.975e-01  7.013e-01]
 [ 0.000e+00  2.220e-16  0.000e+00 -2.197e-02 -4.480e-02 -6.469e-02]
 [ 0.000e+00 -4.528e-16  0.000e+00  6.939e-18  8.080e-04  1.902e-03]
 [ 0.000e+00  4.123e-16  0.000e+00 -1.634e-17  0.000e+00 -1.585e-05]] 

[[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00 -1.110e-16  1.110e-16  1.110e-16 -5.551e-17]
 [ 0.000e+00  0.000e+00  3.331e-16 -2.220e-16 -5.551e-17  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00 -1.110e-16 -1.665e-16  0.000e+00]
 

# II. The need for pivoting

Let's tweak the matrix a little bit, we only change a single element:

In [ ]:
a1 = a.copy()
a1[1, 1] = 3

In [ ]:
np.linalg.matrix_rank(a1)

6

In [ ]:
l, u = diy_lu(a1)

print(l, u)

[[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]] [[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in true_divide
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in matmul
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in matmul
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app


### Test II.1

For a naive LU decomposition to work, all leading minors of a matrix should be non-zero. Check if this requirement is satisfied for the two matrices `a` and `a1`.

(20% of the grade)

In [ ]:
def leading_minors_test(A):
   """
    Function to check all leading Minors of a Matrix not equal to 0
   """
   # ... ENTER YOUR CODE HERE ...

   return True

leading_minors_test(a), leading_minors_test(a1)

(True, False)

### Test II.2

Modify the `diy_lu` routine to implement column pivoting. Keep track of pivots, you can either construct a permutation matrix, or a swap array (your choice).

(40% of the grade)

Implement a function to reconstruct the original matrix from a decompositon. Test your routines on the matrices `a` and `a1`.

(40% of the grade)

# 2. $LU$ factorization column pivoting
When we premultply $A$ by elementary permutation matricex$P_j$( to find a good pivot) and the premultply by lower triangular elemetary matrices $\Lambda_j$  it is trasfrommed in an  upper triangular matrix $U$

$$  \Lambda_{n-1} P_{n-1} \ldots \Lambda_2 P_2 \Lambda_1 P_1 A  = U$$
$$  A  = (\Lambda_{n-1} P_{n-1} \ldots \Lambda_2 P_2 \Lambda_1 P_1)^{-1} U$$

with

\begin{array}{ll}L  &=  (\Lambda_{n-1} P_{n-1} \ldots \Lambda_2 P_2 \Lambda_1 P_1)^{-1}\\
&= P_1^{-1} \Lambda_1^{-1} P_2^{-1} \Lambda_2^{-1} \ldots  P_{n-1}^{-1}\\
 &= P_1^{t} (-\Lambda_1) P_2^{t} (-\Lambda_2) \ldots  P_{n-1}^{t} (-\Lambda_{n-1})
\end{array}



Because the inverse a of a Lower Triangular Elemetary Matrix is minus the matrix $\Lambda_i^{-1} = - \Lambda_i$ and the inverse of a Permutation Matrix (in particular an elementary permutation matrix)  is its transpose [math.stackexchange](
        https://math.stackexchange.com/questions/98549/the-transpose-of-a-permutation-matrix-is-its-inverse#:~:text=Taking%20the%20transpose%20of%20P,Pt%3DP%E2%88%921.)  $P_i^{-1}=P_i^{t}$, so


If we use 
$$  \hat{L}  = (\Lambda_{n-1}  \ldots \Lambda_2\Lambda_1)^{-1}$$
The inverse of a product of matrices is the revesre product of inverses 
$$ \hat{L} = \Lambda_1^{-1}\Lambda_2^{-1} \ldots  \Lambda_{n-1}^{-1} $$
the inverse a of a Lower Triangular Elemetary Matrix is minus the matrix $\Lambda_i^{-1} = - \Lambda_i$
$$ \hat{L} = (-\Lambda_1)  (-\Lambda_2) \ldots  (-\Lambda_{n-1}) $$

Then
$$    A_\pi  = \hat{L} U$$
with $A_\pi$ equal to $A$ with rows permutated acoording to some permutation $\pi$

Equiavlently 
$$   A  = (\hat{L}U)_{\pi'}$$
 $(\hat{L}U)_{\pi'}$ equal to $\hat{L}U$ with rows permutated acoording to some permutation $\pi'$

In [23]:
def diy_lu_column_pivot(A):
    
    #Get the number of rows
    n = A.shape[0]
    
    #Allocate space for P, L, and U
    U = A.copy()
    L = np.eye(n, dtype=np.double)
    P = np.eye(n, dtype=np.double)
    
    #Loop over rows
    for i in range(n):
        
        #Permute rows if needed
        for k in range(i, n): 
            if ~np.isclose(U[i, i], 0.0):
                break
            U[[k, k+1]] = U[[k+1, k]]
            P[[k, k+1]] = P[[k+1, k]]
            
        #Eliminate entries below i with row 
        #operations on U and #reverse the row 
        #operations to manipulate L
        factor = U[i+1:, i] / U[i, i]
        L[i+1:, i] = factor
        U[i+1:] -= factor[:, np.newaxis] * U[i]
        
    return L, U, P

In [24]:
N = 6
a = np.zeros((N, N), dtype=float)
for i in range(N):
    for j in range(N):
        a[i, j] = 3. / (0.6*i*j + 1)
        
L, u, P = diy_lu_column_pivot(a)

print("L\n",L, "\n")
print("u\n",u, "\n")
print("L@u\n", L@u, "\n")
print("a\n",a, "\n")
print("P\n", P, "\n")
print("P@L@u\n", P@L@u, "\n")
print("a\n",a, "\n")

L
 [[1.         0.         0.         0.         0.         0.        ]
 [1.         1.         0.         0.         0.         0.        ]
 [1.         1.45454545 1.         0.         0.         0.        ]
 [1.         1.71428571 1.74223602 1.         0.         0.        ]
 [1.         1.88235294 2.27586207 2.03908376 1.         0.        ]
 [1.         2.         2.67142857 2.944      2.35448132 1.        ]] 

u
 [[ 3.00000000e+00  3.00000000e+00  3.00000000e+00  3.00000000e+00
   3.00000000e+00  3.00000000e+00]
 [ 0.00000000e+00 -1.12500000e+00 -1.63636364e+00 -1.92857143e+00
  -2.11764706e+00 -2.25000000e+00]
 [ 0.00000000e+00  0.00000000e+00  2.62518230e-01  4.57368718e-01
   5.97455283e-01  7.01298701e-01]
 [ 0.00000000e+00  2.22044605e-16  0.00000000e+00 -2.19718086e-02
  -4.48023580e-02 -6.46850044e-02]
 [ 0.00000000e+00 -4.52767547e-16  0.00000000e+00  6.93889390e-18
   8.07981372e-04  1.90237705e-03]
 [ 0.00000000e+00  4.12333415e-16  0.00000000e+00 -1.63374961e-17
   0.0

In [25]:
a1 = a.copy()
a1[1, 1] = 3

L, u , P = diy_lu_column_pivot(a1)

print("L\n",L, "\n")
print("u\n",u, "\n")
print("L@u\n", L@u, "\n")
print("a\n",a, "\n")
print("P\n", P, "\n")
print("P@L@u\n", P@L@u, "\n")
print("a\n",a, "\n")

L
 [[ 1.          0.          0.          0.          0.          0.        ]
 [ 1.          1.          0.          0.          0.          0.        ]
 [ 1.         -0.          1.          0.          0.          0.        ]
 [ 1.          1.17857143 -0.09042748  1.          0.          0.        ]
 [ 1.          1.29411765 -0.15749911  1.63536615  1.          0.        ]
 [ 1.          1.375      -0.20798319  2.06967742  2.08230835  1.        ]] 

u
 [[ 3.00000000e+00  3.00000000e+00  3.00000000e+00  3.00000000e+00
   3.00000000e+00  3.00000000e+00]
 [ 0.00000000e+00 -1.63636364e+00 -2.11764706e+00 -2.34782609e+00
  -2.48275862e+00 -2.57142857e+00]
 [ 0.00000000e+00  0.00000000e+00 -1.63636364e+00 -1.92857143e+00
  -2.11764706e+00 -2.25000000e+00]
 [ 0.00000000e+00  2.22044605e-16  0.00000000e+00  6.14349000e-02
   1.00468556e-01  1.27150425e-01]
 [ 0.00000000e+00 -3.63124230e-16  0.00000000e+00  0.00000000e+00
  -1.82977919e-03 -3.81016447e-03]
 [ 0.00000000e+00  2.96575910e-16  0

In [27]:
a2 = np.array([[4,3,1], [5,7,0], [9,9,3], [8,2,4]])

L, u, P = diy_lu_column_pivot(a2)

print("L\n",L, "\n")
print("u\n",u, "\n")
print("L@u\n", L@u, "\n")
print("a\n",a, "\n")
print("P\n", P, "\n")
print("P@L@u\n", P@L@u, "\n")
print("a\n",a, "\n")

UFuncTypeError: ignored

# 3. $LU$ factorization column pivoting and reconstruction
When we premultply $A$ by elementary permutation matricex$P_j$( to find a good pivot) and the premultply by lower triangular elemetary matrices $\Lambda_j$  it is trasfrommed in an  upper triangular matrix $U$

$$  \Lambda_{n-1} P_{n-1} \ldots \Lambda_2 P_2 \Lambda_1 P_1 A  = U$$
$$  A  = (\Lambda_{n-1} P_{n-1} \ldots \Lambda_2 P_2 \Lambda_1 P_1)^{-1} U$$
$$  A  = (\Lambda_{n-1} P_{n-1} \ldots \Lambda_2 P_2 \Lambda_1 P_1)^{-1} U$$

The inverse of a product of matrices is the revesre product of inverses 
$$  A  = ( P_1^{-1} \Lambda_1^{-1} P_2^{-1} \Lambda_2^{-1} \ldots  P_{n-1}^{-1}\Lambda_{n-1}^{-1} ) U$$

So the $LU$ column pivot factorization is 
$$  A  = L U$$
with
$$ U = \Lambda_{n-1} P_{n-1} \ldots \Lambda_2 P_2 \Lambda_1 P_1 A  $$



$$ L  = P_1^{-1} \Lambda_1^{-1} P_2^{-1} \Lambda_2^{-1} \ldots  P_{n-1}^{-1}\Lambda_{n-1}^{-1} $$

the inverse a of a Lower Triangular Elemetary Matrix is minus the matrix $\Lambda_i^{-1} = - \Lambda_i$ and the inverse of a Permutation Matrix (in particular an elementary permutation matrix)  is its transpose [math.stackexchange](
        https://math.stackexchange.com/questions/98549/the-transpose-of-a-permutation-matrix-is-its-inverse#:~:text=Taking%20the%20transpose%20of%20P,Pt%3DP%E2%88%921.)  $P_i^{-1}=P_i^{t}$, so

$$ L  =  P_1^{t} (-\Lambda_1) P_2^{t} (-\Lambda_2) \ldots  P_{n-1}^{t} (-\Lambda_{n-1}) $$


In [28]:
def diy_lu_column_pivot_reconstruct(A):
    
    #Get the number of rows
    n = A.shape[0]
    
    U = A.copy()
    L = np.eye(n, dtype=np.double)
    
    #Loop over rows
    for i in range(n):
            
        #Eliminate entries below i with row operations 
        #on U and reverse the row operations to 
        #manipulate L
        factor = U[i+1:, i] / U[i, i]
        L[i+1:, i] = factor
        U[i+1:] -= factor[:, np.newaxis] * U[i]
        
    return L, U

In [29]:
N = 6
a = np.zeros((N, N), dtype=float)
for i in range(N):
    for j in range(N):
        a[i, j] = 3. / (0.6*i*j + 1)

L, u = diy_lu_column_pivot_reconstruct(a)

print("L\n",L, "\n")
print("u\n",u, "\n")
print("L@u\n", L@u, "\n")
print("a\n",a, "\n")

L
 [[1.         0.         0.         0.         0.         0.        ]
 [1.         1.         0.         0.         0.         0.        ]
 [1.         1.45454545 1.         0.         0.         0.        ]
 [1.         1.71428571 1.74223602 1.         0.         0.        ]
 [1.         1.88235294 2.27586207 2.03908376 1.         0.        ]
 [1.         2.         2.67142857 2.944      2.35448132 1.        ]] 

u
 [[ 3.00000000e+00  3.00000000e+00  3.00000000e+00  3.00000000e+00
   3.00000000e+00  3.00000000e+00]
 [ 0.00000000e+00 -1.12500000e+00 -1.63636364e+00 -1.92857143e+00
  -2.11764706e+00 -2.25000000e+00]
 [ 0.00000000e+00  0.00000000e+00  2.62518230e-01  4.57368718e-01
   5.97455283e-01  7.01298701e-01]
 [ 0.00000000e+00  2.22044605e-16  0.00000000e+00 -2.19718086e-02
  -4.48023580e-02 -6.46850044e-02]
 [ 0.00000000e+00 -4.52767547e-16  0.00000000e+00  6.93889390e-18
   8.07981372e-04  1.90237705e-03]
 [ 0.00000000e+00  4.12333415e-16  0.00000000e+00 -1.63374961e-17
   0.0

In [32]:
a1 = a.copy()
a1[1, 1] = 3

L, u, = diy_lu_column_pivot_reconstruct(a1)

print("L\n",L, "\n")
print("u\n",u, "\n")
print("L@u\n", L@u, "\n")
print("a2\n",a1, "\n")

# Quick sanity check: L times U must equal the original matrix, up to floating-point errors.
print("L@u - a1\n",L@u - a1, "\n")

L
 [[  1.   0.   0.   0.   0.   0.]
 [  1.   1.   0.   0.   0.   0.]
 [  1. -inf   1.   0.   0.   0.]
 [  1. -inf  nan   1.   0.   0.]
 [  1. -inf  nan  nan   1.   0.]
 [  1. -inf  nan  nan  nan   1.]] 

u
 [[ 3.          3.          3.          3.          3.          3.        ]
 [ 0.          0.         -1.63636364 -1.92857143 -2.11764706 -2.25      ]
 [        nan         nan        -inf        -inf        -inf        -inf]
 [        nan         nan         nan         nan         nan         nan]
 [        nan         nan         nan         nan         nan         nan]
 [        nan         nan         nan         nan         nan         nan]] 

L@u
 [[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]] 

a2
 [[3.         3.         3.         3.         3.         3.        ]
 [3.         3.         1.36363636 1.07142857 0.88235294 0.75      ]
 [3.         1.36363636 0.88

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in true_divide
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in multiply
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in matmul
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in matmul
  if sys.path[0] == '':


In [34]:
a2 = np.array([[4,3,1], [5,7,0], [9,9,3], [8,2,4]])

L, u, = diy_lu_column_pivot_reconstruct(a2)

print("L\n",L, "\n")
print("u\n",u, "\n")
print("L@u\n", L@u, "\n")
print("a2\n",a2, "\n")

# Quick sanity check: L times U must equal the original matrix, up to floating-point errors.
print("L@u - a2\n",L@u - a2, "\n")

UFuncTypeError: ignored